# Create Synset Dataset

In [1]:
from nltk.corpus import wordnet
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import sys
sys.path.append(r'C:\Temp\SoundOfAI\rg_text_to_sound\tts_pipeline\src')
import ipdb
import pandas as pd
import numpy as np
from tqdm import tqdm
from match_word_to_words import prepare_dataset,word_to_wordpair_estimator,word_to_words_matcher,prepare_dataset
import matplotlib.pyplot as plt
cb = wordnet.synset('cookbook.n.01')


In [2]:
k=20 #maximum number of synonyms we might need

In [3]:
target_word_pairs = [('bright', 'dark'), ('full', 'hollow'),( 'smooth', 'rough'), ('warm', 'metallic'), ('clear', 'muddy'), ('thin', 'thick'), ('pure', 'noisy'), ('rich', 'sparse'), ('soft', 'hard')]
target_words = np.array(target_word_pairs).ravel().tolist()

In [4]:
wordnet.synset?

Signature: wordnet.synset(name)
Docstring: <no docstring>
File:      c:\users\tob\anaconda31\envs\nlp\lib\site-packages\nltk\corpus\reader\wordnet.py
Type:      method


In [5]:
target_word = target_words[0]


In [6]:
def get_synonyms(target_word):
    syns = wordnet.synsets(target_word)
    mys = set()
    for syn in syns:
        #print(syn.name())
        mys |=set([m.name() for m in syn.lemmas()])
    return mys

def tolist(myset,length):
    l = list(myset)
    if len(l)<length:
        l+=['' for i in range(length-len(myset))]
        return l
    else:
        return l[:length]
    
    

In [7]:
df = pd.DataFrame([pd.Series(tolist(get_synonyms(targetword),k) ,name = targetword) for targetword in target_words]).T

In [8]:
df_sorted = df.where(~df.isin(df.columns),'').apply(lambda x:sorted(x,reverse=True))

In [9]:
df_sorted

,bright,dark,full,hollow,smooth,rough,warm,metallic,clear,muddy,thin,thick,pure,noisy,rich,sparse,soft,hard
0,vivid,wickedness,wide-cut,vacuous,unruffled,unsmooth,warmly,metal,top,waterlogged,thinly,thickset,virtuous,,robust,,voiced,voiceless
1,undimmed,sullen,wide,hollow_out,tranquil,scratchy,warm_up,,take_in,turbid,thin_out,thickly,virginal,,rich_people,,subdued,unvoiced
2,smart,sorry,wax,holler,suave,roughly,tender,,straighten_out,swampy,tenuous,thickheaded,virgin,,racy,,sonant,tough
3,shiny,sinister,total,hole,still,rough_out,strong,,sort_out,squashy,slim_down,thick-skulled,vestal,,productive,,piano,toilsome
4,shining,shadow,to_the_full,excavate,smoothen,rough_in,quick,,solve,soggy,slim,stocky,utter,,plentiful,,mild,surd
5,promising,saturnine,replete,empty,smooth_out,rocky,lovesome,,shed_light_on,sloughy,slight,slurred,unadulterated,,plenteous,,lenient,strong
6,lustrous,nighttime,good,dig,shine,raspy,fond,,sack,sloppy,slenderize,midst,thoroughgoing,,full-bodied,,indulgent,severely
7,hopeful,moody,fully,core_out,quiet,rasping,ardent,,realize,quaggy,slender,loggerheaded,stark,,fertile,,gentle,severe
8,burnished,iniquity,full_phase_of_the_moon,,politic,pugnacious,affectionate,,net,murky,reduce,heavyset,staring,,fat,,flaccid,laborious
9,brilliantly,grim,full_moon,,polish,jumpy,,,make,muddy_up,melt_off,fatheaded,sodding,,deep,,flabby,knockout


In [10]:
df = pd.read_csv('text_to_qualities.csv')
dfnew = prepare_dataset(df)
dfnew[dfnew.description.str.match('\'')]
dfnew['description'] = dfnew.description.str.replace("'","")
dfnew['description']=dfnew.description.str.lower().str.replace('(\(not.*\))','',regex=True)
dfnew = dfnew[~dfnew.description.str.match('\(.*\)')]
df_score = dfnew.iloc[:,1:]
dfnew.head()

,description,bright_vs_dark,full_vs_hollow,smooth_vs_rough,warm_vs_metallic,clear_vs_muddy,thin_vs_thick,pure_vs_noisy,rich_vs_sparse,soft_vs_hard
0,reversed,59.0,74.0,42.0,44.0,75.0,69.0,60.0,85.0,58.0
3,acoustic,47.0,12.5,69.0,42.0,42.0,59.5,45.0,27.0,12.0
4,aggressive,20.0,50.0,99.0,84.0,77.0,77.0,79.0,23.0,79.0
5,artificial,50.0,50.0,99.0,70.0,50.0,50.0,76.0,50.0,50.0
6,biting,23.0,46.0,36.0,29.0,58.0,50.0,63.5,42.0,72.5


In [11]:
wordpairnames = df_score.columns.tolist()

In [12]:
wordpairnames_to_wordpair_dict = {s:t for s,t in zip(wordpairnames,target_word_pairs)}

In [13]:
df_sorted.to_excel('Training_dataset.xlsx',index=False)

In [14]:
target_word_pairs

[('bright', 'dark'),
 ('full', 'hollow'),
 ('smooth', 'rough'),
 ('warm', 'metallic'),
 ('clear', 'muddy'),
 ('thin', 'thick'),
 ('pure', 'noisy'),
 ('rich', 'sparse'),
 ('soft', 'hard')]

In [15]:
df_sorted = pd.read_excel('Training_dataset.xlsx',index_col=None)
df_sorted

,bright,dark,full,hollow,smooth,rough,warm,metallic,clear,muddy,thin,thick,pure,noisy,rich,sparse,soft,hard
0,vivid,wickedness,wide-cut,vacuous,unruffled,unsmooth,warmly,metal,top,waterlogged,thinly,thickset,virtuous,NaN,robust,NaN,voiced,voiceless
1,undimmed,sullen,wide,hollow_out,tranquil,scratchy,warm_up,NaN,take_in,turbid,thin_out,thickly,virginal,NaN,rich_people,NaN,subdued,unvoiced
2,smart,sorry,wax,holler,suave,roughly,tender,NaN,straighten_out,swampy,tenuous,thickheaded,virgin,NaN,racy,NaN,sonant,tough
3,shiny,sinister,total,hole,still,rough_out,strong,NaN,sort_out,squashy,slim_down,thick-skulled,vestal,NaN,productive,NaN,piano,toilsome
4,shining,shadow,to_the_full,excavate,smoothen,rough_in,quick,NaN,solve,soggy,slim,stocky,utter,NaN,plentiful,NaN,mild,surd
5,promising,saturnine,replete,empty,smooth_out,rocky,lovesome,NaN,shed_light_on,sloughy,slight,slurred,unadulterated,NaN,plenteous,NaN,lenient,strong
6,lustrous,nighttime,good,dig,shine,raspy,fond,NaN,sack,sloppy,slenderize,midst,thoroughgoing,NaN,full-bodied,NaN,indulgent,severely
7,hopeful,moody,fully,core_out,quiet,rasping,ardent,NaN,realize,quaggy,slender,loggerheaded,stark,NaN,fertile,NaN,gentle,severe
8,burnished,iniquity,full_phase_of_the_moon,NaN,politic,pugnacious,affectionate,NaN,net,murky,reduce,heavyset,staring,NaN,fat,NaN,flaccid,laborious
9,brilliantly,grim,full_moon,NaN,polish,jumpy,NaN,NaN,make,muddy_up,melt_off,fatheaded,sodding,NaN,deep,NaN,flabby,knockout


In [16]:

for ind,row in df_sorted.T.iterrows():
    target_word = row.name
    test_words = pd.Series([x for x in row if x!=''])
    print(target_word,':',','.join(test_words.dropna().tolist()))

bright : vivid,undimmed,smart,shiny,shining,promising,lustrous,hopeful,burnished,brilliantly,brilliant,brightly
dark : wickedness,sullen,sorry,sinister,shadow,saturnine,nighttime,moody,iniquity,grim,glum,drear,drab,dour,dismal,disconsolate,darkness,dark-skinned,blue
full : wide-cut,wide,wax,total,to_the_full,replete,good,fully,full_phase_of_the_moon,full_moon,full-of-the-moon,entire,broad
hollow : vacuous,hollow_out,holler,hole,excavate,empty,dig,core_out
smooth : unruffled,tranquil,suave,still,smoothen,smooth_out,shine,quiet,politic,polish,placid,liquid,legato,fluid,fluent,bland
rough : unsmooth,scratchy,roughly,rough_out,rough_in,rocky,raspy,rasping,pugnacious,jumpy,jolting,harsh,gravelly,grating,fierce,crude,bumpy,approximative,approximate
warm : warmly,warm_up,tender,strong,quick,lovesome,fond,ardent,affectionate
metallic : metal
clear : top,take_in,straighten_out,sort_out,solve,shed_light_on,sack,realize,net,make,exculpate,elucidate,discharge,crystallise,crystalize,crystalise,clea

In [17]:
lang_model = 'en_core_web_lg'

w2wpe = word_to_wordpair_estimator()
w2wpe.build(wordpairnames,target_word_pairs,lang_model=lang_model)

acc_dict = dict()
cm_dict = dict()
print('Mean accuracy over Synset words:')
for wordpairname,targetwordpair in zip(wordpairnames,target_word_pairs):
    leftwords = df_sorted[targetwordpair[0]]
    leftwords = leftwords[leftwords!=''].dropna()
    rightwords = df_sorted[targetwordpair[1]]
    rightwords = rightwords[rightwords!=''].dropna()
    
    leftpred = [w2wpe.match_word_to_wordpair(word,wordpairname,verbose=False)['closest word'] for word in leftwords]
    if len(leftpred)>0:
        bool_left = np.array(leftpred)==targetwordpair[0]
    else:
        bool_left = np.array([])
    rightpred = [w2wpe.match_word_to_wordpair(word,wordpairname,verbose=False)['closest word'] for word in rightwords]
    if len(rightpred)>0:
        bool_right = np.array(rightpred)==targetwordpair[1]
    else:
        bool_right = np.array([])
    correct_l = np.sum(bool_left)
    correct_r = np.sum(bool_right)
    y = np.hstack((bool_left,bool_right))    
    accuracy = np.mean(y)
    print(f'{wordpairname:<20}:{accuracy:5.2f} ({leftwords.size:2} "{targetwordpair[0]}" words, {rightwords.size:2} "{targetwordpair[1]}" words) ({correct_l}/{correct_r} correct) ')
    acc_dict[wordpairname]=accuracy
    cm = confusion_matrix(leftpred,np.array([targetwordpair[0] for i in range(len(leftpred))]),labels=targetwordpair)+confusion_matrix(rightpred,np.array([targetwordpair[1] for i in range(len(rightpred))]),labels=targetwordpair)
    cm_dict[wordpairname]=cm
accuracy = pd.Series(acc_dict)

Mean accuracy over Synset words:
bright_vs_dark      : 0.87 (12 "bright" words, 19 "dark" words) (12/15 correct) 
full_vs_hollow      : 0.71 (13 "full" words,  8 "hollow" words) (13/2 correct) 
smooth_vs_rough     : 0.71 (16 "smooth" words, 19 "rough" words) (6/19 correct) 
warm_vs_metallic    : 1.00 ( 9 "warm" words,  1 "metallic" words) (9/1 correct) 
clear_vs_muddy      : 0.87 (20 "clear" words, 19 "muddy" words) (20/14 correct) 
thin_vs_thick       : 0.77 (16 "thin" words, 19 "thick" words) (16/11 correct) 
pure_vs_noisy       : 0.89 (18 "pure" words,  0 "noisy" words) (16/0.0 correct) 
rich_vs_sparse      : 0.50 (12 "rich" words,  0 "sparse" words) (6/0.0 correct) 
soft_vs_hard        : 0.84 (17 "soft" words, 20 "hard" words) (11/20 correct) 


In [19]:
accuracy

bright_vs_dark      0.870968
full_vs_hollow      0.714286
smooth_vs_rough     0.714286
warm_vs_metallic    1.000000
clear_vs_muddy      0.871795
thin_vs_thick       0.771429
pure_vs_noisy       0.888889
rich_vs_sparse      0.500000
soft_vs_hard        0.837838
dtype: float64

In [18]:
for key, val in cm_dict.items():
    print(key)
    print(val)

bright_vs_dark
[[12  4]
 [ 0 15]]
full_vs_hollow
[[13  6]
 [ 0  2]]
smooth_vs_rough
[[ 6  0]
 [10 19]]
warm_vs_metallic
[[9 0]
 [0 1]]
clear_vs_muddy
[[20  5]
 [ 0 14]]
thin_vs_thick
[[16  8]
 [ 0 11]]
pure_vs_noisy
[[16  0]
 [ 2  0]]
rich_vs_sparse
[[6 0]
 [6 0]]
soft_vs_hard
[[11  0]
 [ 6 20]]
